In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from textblob import TextBlob

In [2]:
data = {'Name': ['Poon Dustin', 'Wong Esther', 'Horstead Matt'],
        'Student ID': ['301389544', "301328854", '301350353'],
        'Section Contributed': ['ALL','-','-'],
        'Section Edited': ['ALL','-','-'],
        'Other Contribution': ['','','']}
table = pd.DataFrame(data, columns=['Name', 'Student ID', 'Section Contributed', 'Section Edited', 'Other Contribution'])
table

,Name,Student ID,Section Contributed,Section Edited,Other Contribution
0,Poon Dustin,301389544,ALL,ALL,
1,Wong Esther,301328854,-,-,
2,Horstead Matt,301350353,-,-,


In [3]:
mySession = requests.session()

response = mySession.get("https://www.rottentomatoes.com/m/minnal_murali/reviews")
soup = bs(response.content, 'html.parser')
    
# we find the table containing the reviews
articleFeed = soup.findAll("div", {"class": "the_review"})


I used python module "textblob" to run sentimental analysis to determine the reviewer's polarity. (whether it is postive, negative or neutral valued). I have also analysed the accuracy based on the review given.    
The data used is from the movie review of "MINNAL MURALI" on RottenTomatos.
SentimentalAnalysis is done on "All critics" section.


In [4]:
textAll = [review.get_text() for review in articleFeed] #retrieve only text 
articleText = [x.strip() for x in textAll]              #remove /r/n
zens = 0                                                #restet the sentence value
mylist=[]                                               #create a new list
for reviews in articleText:    
    ind = (articleText[zens])                           # for each reviews add one utill reaches reviews in article text 
    zens = zens+1                         
    strSen = TextBlob(ind)                              #turing sentence into a textblob format
    mylist.append(round(strSen.sentiment.polarity,2))   #adding it into the list

#I am doing it this way because textblob takes analysis for each sentence, so I would get more output than needed.
#I want an analysis for each review not each sentence

In [5]:
print(mylist)

[0.35, 0.0, 0.5, 0.41, 0.14, -0.12, 0.1]


In [6]:
value = []
for i in (mylist):
    if (i > 0):   
        value.append("Positive")
    elif (i < 0):
        value.append("Negative")
    else:
        value.append("Neutral")
#this is for setting a tag for each reviews    

In [7]:
print(value)

['Positive', 'Neutral', 'Positive', 'Positive', 'Positive', 'Negative', 'Positive']


In [8]:
#to put it in a pandas table
reviewTable = soup.find("div", {"class": "review_table"})
reviewers = [r.get_text() for r in reviewTable.select(".review_table_row .critic_name .articleLink")]
reviewText = [t.get_text() for t in reviewTable.select(".review_table_row .the_review")]
reviewText = [x.strip() for x in reviewText]

reviews = {'Reviewer':reviewers, 'Text':reviewText, 'Polarity':mylist, 'Est.Value':value}

reviews_df = pd.DataFrame(reviews) 

reviews_df

,Reviewer,Text,Polarity,Est.Value
0,Brian Costello,This is an epic superhero movie that deftly mi...,0.35,Positive
1,Shah Shahid,Minnal Murali adds to the discourse of superhe...,0.00,Neutral
2,Rohan Naahar,"Not all heroes wear capes, they say. But they ...",0.50,Positive
3,Akhil Arora,In the hands of a more astute director and wit...,0.41,Positive
4,Anna M.M. Vetticad,Minnal Murali film may be a tribute to the Hol...,0.14,Positive
5,Saibal Chatterjee,Jaison/Minnal Murali is played with a blend of...,-0.12,Negative
6,Shubhra Gupta,"While the film is aware of its superhero-ness,...",0.10,Positive


In [11]:
print('Accuracy 72% of comparing between the polarity of textblob and the actual ratings given. This is the reason mainly because the codes only run sentimental analysis on the short description of the review and not the full review. Otherwise, comparing to the "preview text", the polarity is very accurate.')

Accuracy 72% of comparing between the polarity of textblob and the actual ratings given. This is the reason mainly because the codes only run sentimental analysis on the short description of the review and not the full review. Otherwise, comparing to the "preview text", the polarity is very accurate.


                                                  References

Taboada, M. (2022, February 17). sda-250-sfu/Lab4_tips. GitHub. https://github.com/sda-250-

    sfu/Lab4_tips/blob/main/Lab4_solutions.ipynb

The Greatest Showman - Movie Reviews. (n.d.). Www.rottentomatoes.com. Retrieved March 18, 2022, from 
    
    https://www.rottentomatoes.com/m/the_greatest_showman_2017/reviews

Tutorial: Quickstart — TextBlob 0.16.0 documentation. (n.d.). Textblob.readthedocs.io. 
    
    https://textblob.readthedocs.io/en/dev/quickstart.html

In [9]:
#pip install textblob